### Load Model and Config

In [ ]:
from omegaconf import OmegaConf
import sys
sys.path.append('..')
from experiments.mesa_ppg_bss import Experiment

# model and config download is located here: https://drive.google.com/file/d/1z4gHzVAsF3FKio2O0_7h1NTSYMe7f4EO/view?usp=sharing

# load model
EXPR_PATH = '../checkpoints/conv_meae_ppg.ckpt'
CONFIG_PATH = '../checkpoints/config.yaml'

config = OmegaConf.load(CONFIG_PATH)
print(config)
experiment = Experiment
experiment = experiment.load_from_checkpoint(checkpoint_path=EXPR_PATH, config=config.experiment_config)

model = experiment.model
model.eval()

{'experiment_config': {'model': ['models.multi_cnne_cnn_ae_1d', 'ConvolutionalAutoencoder'], 'lr': 0.001, 'lr_step_size': 20, 'weight_decay': 1e-06, 'gradient_clip_val': 0.5, 'z_loss': True, 'z_lr': 0.01, 'sep_loss': True, 'sep_norm': 'L1', 'sep_lr': 0.05, 'zero_loss': True, 'zero_lr': 0.1, 'use_weight_norm': True, 'zero_mixing_weights': True, 'use_highpass': False, 'low_freq_lim': 0.64, 'use_lowpass': False, 'high_freq_lim': 3, 'channels': [32, 32, 64, 64, 128, 128, 256, 256], 'num_encoders': 8, 'hidden': 256, 'norm_type': 'group_norm', 'max_epochs': 150, 'batch_size': 128, 'train_val_split': 0.8, 'dataloader': ['utils.dataloader.mesa_ppg_bss', 'DataModule'], 'data_path': './data/mesa_ppg_ecg/', 'num_workers': 0, 'input_channels': 1, 'input_padding': 0, 'input_signal_type': 'PPG', 'fs': 125, 'signal_duration': 48, 'padding': 144, 'get_ecg': False, 'use_upsampled_hr': True, 'monitor': 'recon_loss/train_step', 'monitor_mode': 'min', 'save_plots': True, 'plot_dir': './plots/', 'plot_step

/home/webster/.virtualenvs/torch_260/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


ConvolutionalAutoencoder(
  (encoders): ModuleList(
    (0-7): 8 x ConvolutionalEncoder(
      (encoder): Sequential(
        (0): EncoderBlock(
          (block): Sequential(
            (0): Conv1d(1, 4, kernel_size=(7,), stride=(1,), padding=(3,))
            (1): ReLU(inplace=True)
            (2): BatchNorm1d(4, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
            (3): Conv1d(4, 4, kernel_size=(7,), stride=(1,), padding=(3,))
            (4): ReLU(inplace=True)
            (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (6): BatchNorm1d(4, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
          )
        )
        (1): EncoderBlock(
          (block): Sequential(
            (0): Conv1d(4, 4, kernel_size=(7,), stride=(1,), padding=(3,))
            (1): ReLU(inplace=True)
            (2): BatchNorm1d(4, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
            (3): Conv1d(4, 4, k

In [ ]:
import torch

ppg = None # load 1-dimensional array example here with shape divisible by 2^8

x = torch.tensor(ppg, dtype=torch.float32).reshape(1, 1, len(ppg))

# simple min-max scaling for data
x_max = torch.max(x, dim=-1, keepdim=True)[0]
x_min = torch.min(x, dim=-1, keepdim=True)[0]
x = (x - x_min) / (x_max - x_min)

with torch.no_grad():
    z = model.encode(x)
    x_pred = torch.sigmoid_(model.decode(z)) # <- this is the full reconstruction of the input signal
    y = []
    for z_a in range(len(z)):
        z_pad = [torch.zeros_like(z[0]) for _ in range(len(z))]
        z_pad[z_a] = z[z_a]
        # <- here you can also add specfic sources by indexing z_a, e.g. z_pad[3] = z[3]
        y_i = model.decode(z_pad)
        y_i = torch.sigmoid(y_i).squeeze().cpu().numpy()
        y.append(y_i)

# y contains all individual sources. For the model provided by the authors 
# the heart source has index 1 (i.e. y[0]).